# Stack Overflow

## Introduction 

In this assignment, we will look at some posts on Stack Overflow during the year of 2015 and measure the similarity of users by looking at the types of questions they answer. Do not delete the output of your code cells. This assignment is to be completed **INDIVIDUALLY** and it is due on **September 30**.

Please update the README with your BU username.

## Step 0. Preparation

Before we start working on the notebook, let's make sure that everything is setup properly. You should have downloaded and installed
* [Anaconda](https://store.continuum.io/cshop/anaconda/)
* [Git](http://git-scm.com/downloads)

If you are working from the undergraduate lab (on a linux machine) these are both installed, but you need to follow the instructions [from here](https://github.com/mcrovella/CS505-Computational-Tools-for-Data-Science/blob/master/2-Getting-Started.ipynb).



## Step 1. Getting the data

Let's make a sample request to retrieve some questions posted on Stack Exchange on the first day of 2015. Documentation of the Stack Exchange API can be found [here](https://api.stackexchange.com/docs).

In [ ]:
import requests
from datetime import datetime

start_time = 1420070400 # 01-01-2015 at 00:00:00
end_time   = 1420156800 # 01-02-2015 at 00:00:00

response = requests.get("https://api.stackexchange.com/2.2/questions?pagesize=100" +
                        "&fromdate=" + str(start_time) + "&todate=" + str(end_time) +
                        "&order=asc&sort=creation&site=stackoverflow")
print(response)

All dates in the Stack Exchange API are in [unix epoch time](https://en.wikipedia.org/wiki/Unix_time). The format for the request string is specified [here](https://api.stackexchange.com/docs/questions).

We can try to print the response that Stack Exchange returns.

In [ ]:
print(response.text)

It is not possible to read the raw response. Instead, we need to decode the raw response as JSON and use the `json` library to print it.

In [ ]:
import json

json_response = response.json()

print(json.dumps(json_response, indent=2))

Now we can easily see that the response consists of a list of question items. For each of these items, we get information about its attributes such as its `creation_date`, `answer_count`, `owner`, `title`, etc.

Notice that has_more is true. To get more items, we can [request the next page](https://api.stackexchange.com/docs/paging).

-----------------

## Step 2. Parsing the responses

In this section, we practice some of the basic Python tools that we learned in class and the powerful string handling methods that Python offers. Our goal is to be able to pick the interesting parts of the response and transform them in a format that will be useful to us.

First let's isolate the creation_date in the response. Fill in the rest of the ```print_creation_dates_json()``` function that reads the response and prints the creation dates. Notice that a JSON object is basically a dictionary. **(5 pts)**

In [3]:
def print_creation_dates_json(response):
    """
    Prints the creation_date of all the questions in the response.
    
    Parameters:
        response: Response object
    """
    json_response=response.json()
    for i in range(0, len(json_response["items"])):
		print(json_response["items"][i]['creation_date'])
    

Write the code that calls the ```print_creation_dates_json()``` function to print out all the creation dates of questions posted on the first day in 2015. Please be aware of Stack Exchange's [rate limit](https://api.stackexchange.com/docs/throttle). **(5 pts)**

In [ ]:
start_time = 1420070400 # 01-01-2015 at 00:00:00
end_time   = 1420156800 # 01-02-2015 at 00:00:00


i=1
response = requests.get("https://api.stackexchange.com/2.2/questions?page="+str(i)+"&pagesize=100" + "&fromdate=" + str(start_time) + "&todate=" + str(end_time) +"&order=asc&sort=creation&site=stackoverflow")
json_response = response.json()
while(json_response["has_more"]):
	
	response = requests.get("https://api.stackexchange.com/2.2/questions?page="+str(i)+"&pagesize=100" + "&fromdate=" + str(start_time) + "&todate=" + str(end_time) +"&order=asc&sort=creation&site=stackoverflow")
	print_creation_dates_json(response)
	time.sleep(3)
	i+=1

Due to time constraints, we have downloaded the [data dump](https://drive.google.com/open?id=0B4bdB5WPHGsqTm9PZlBhRVQzRmc) for Stack Overflow's posts in 2015. The link is only visible to BU students, so you must be logged in to your BU email. Note that the XML file is 10GB. If you don't have space on your computer, you can download it into `/scratch` on one of the machines in the undergrad lab or you can download it onto a USB. You may want to work with a subset of this data at first, but your solution should be efficient enough to work with the whole dataset. For example, if you call `read()` on the whole dataset, you will get a `MemoryError`.

Do not commit the data file. You may assume that we will place the data file in the same directory as your IPython Notebook, so provide a relative path when loading the data file.

Write a function to parse out the questions posted in 2015. These are posts with `PostTypeId=1`. Make a `pandas DataFrame` with 4 columns: `Id`, `CreationDate`, `OwnerUserId`, and the first tag in `Tags`. Save the `DataFrame` to a file named `question_dataframe.csv` using `to_csv()`. You may also parse out the data to `question_dataframe.csv` first and then create the `pandas DataFrame` using `read_csv()`. **(10 pts)**

Your code should run in a few minutes (not hours!). In a markdown cell, write down the approximate run time of your code.

In [ ]:
tree = ET.parse('stackoverflow-posts-2015.xml')
root = tree.getroot()
data =[]
for child in root:
	if(child.attrib['PostTypeId']=='1'):
		row =[]
		if ('Id' in child.attrib and 'CreationDate' in child.attrib and 'OwnerUserId' in child.attrib and 'Tags' in child.attrib):
			row.extend([child.attrib['Id'],child.attrib['CreationDate'],child.attrib['OwnerUserId']])
			row.extend([child.attrib['Tags'][1:child.attrib['Tags'].find('>')]])
			data.append(row)


df=pandas.DataFrame({'Id':[row[0] for row in data],'CreationDate':[row[1] for row in data],'OwnerUserId':[row[2] for row in data],'Tags':[row[3] for row in data]})

df.to_csv('question_dataframe.csv')

-----------------

## Step 3. Putting it all together

We are now ready to tackle our original problem. Write a function to measure the similarity of the top 100 users with the most answer posts. Compare the users based on the types of questions they answer. We will categorize a question by its first tag. You may choose to implement any one of the similarity/distance measures we discussed in class. **(30pts)**

Note that answers are posts with `PostTypeId=2`. The ID of the question in answer posts is the `ParentId`.

You may find the [sklearn.feature_extraction](http://scikit-learn.org/stable/modules/feature_extraction.html) module helpful.

In [ ]:
tree = ET.parse('stackoverflow-posts-2015.xml')
root = tree.getroot()

date=[]
for child in root:
	if(child.attrib['PostTypeId']=='2'):
		row=[]
		if('ParentId' in child.attrib):
			if('Id' in child.attrib):
				if('CreationDate' in child.attrib):
					if('OwnerUserId' in child.attrib):
						row.extend([child.attrib['Id'],child.attrib['CreationDate'],child.attrib['OwnerUserId'],child.attrib['ParentId']])
						data.append(row)
dfq=pd.DataFrame({'Id':[row[0] for row in data],'CreationDate':[row[1] for row in data],'OwnerUserId':[row[2] for row in data],'ParentId':[row[3] for row in data]})
dfq.to_csv('answer_dataframe.csv')
df['OwnerUserId']=df['OwnerUserId'].astype('int')
df.merge(dfq, left_on='OwnerUserId', right_on='OwnerUserId',how='inner')
df.groupby('OwnerUserId').count()['Id'].nlargest(100)


Plot the distance of the top 100 users using a [heatmap](https://stanford.edu/~mwaskom/software/seaborn/generated/seaborn.heatmap.html). **(10 pts)**

-----------------